In [1]:
#Importing Libraries
import pandas as pd
import numpy as np
from math import radians, sin, cos, sqrt, atan2


In [2]:
#Loading the Datasets
df_house = pd.read_csv("NY-House-Dataset (2).csv")
df_mta = pd.read_csv("MTA_Subway_Stations_20251120.csv")



In [5]:
import pandas as pd

url = "https://data.cityofnewyork.us/resource/qgea-i56i.json?$limit=2000"
df_crime = pd.read_json(url)
df_crime.head()
df_crime.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 40 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   cmplnt_num                   2000 non-null   object 
 1   cmplnt_fr_dt                 2000 non-null   object 
 2   cmplnt_fr_tm                 2000 non-null   object 
 3   cmplnt_to_dt                 1908 non-null   object 
 4   cmplnt_to_tm                 2000 non-null   object 
 5   addr_pct_cd                  2000 non-null   int64  
 6   rpt_dt                       2000 non-null   object 
 7   ky_cd                        2000 non-null   int64  
 8   ofns_desc                    2000 non-null   object 
 9   pd_cd                        1998 non-null   float64
 10  pd_desc                      2000 non-null   object 
 11  crm_atpt_cptd_cd             2000 non-null   object 
 12  law_cat_cd                   2000 non-null   object 
 13  boro_nm           

In [6]:
# Make column names lowercase and snake_case
df_house.columns = df_house.columns.str.lower().str.strip().str.replace(" ", "_")

df_house.head()


,brokertitle,type,price,beds,bath,propertysqft,address,state,main_address,administrative_area_level_2,locality,sublocality,street_name,long_name,formatted_address,latitude,longitude
0,Brokered by Douglas Elliman -111 Fifth Ave,Condo for sale,315000,2,2.000000,1400.0,2 E 55th St Unit 803,"New York, NY 10022","2 E 55th St Unit 803New York, NY 10022",New York County,New York,Manhattan,East 55th Street,Regis Residence,"Regis Residence, 2 E 55th St #803, New York, N...",40.761255,-73.974483
1,Brokered by Serhant,Condo for sale,195000000,7,10.000000,17545.0,Central Park Tower Penthouse-217 W 57th New Yo...,"New York, NY 10019",Central Park Tower Penthouse-217 W 57th New Yo...,United States,New York,New York County,New York,West 57th Street,"217 W 57th St, New York, NY 10019, USA",40.766393,-73.980991
2,Brokered by Sowae Corp,House for sale,260000,4,2.000000,2015.0,620 Sinclair Ave,"Staten Island, NY 10312","620 Sinclair AveStaten Island, NY 10312",United States,New York,Richmond County,Staten Island,Sinclair Avenue,"620 Sinclair Ave, Staten Island, NY 10312, USA",40.541805,-74.196109
3,Brokered by COMPASS,Condo for sale,69000,3,1.000000,445.0,2 E 55th St Unit 908W33,"Manhattan, NY 10022","2 E 55th St Unit 908W33Manhattan, NY 10022",United States,New York,New York County,New York,East 55th Street,"2 E 55th St, New York, NY 10022, USA",40.761398,-73.974613
4,Brokered by Sotheby's International Realty - E...,Townhouse for sale,55000000,7,2.373861,14175.0,5 E 64th St,"New York, NY 10065","5 E 64th StNew York, NY 10065",United States,New York,New York County,New York,East 64th Street,"5 E 64th St, New York, NY 10065, USA",40.767224,-73.969856


In [7]:
# Convert key numeric columns
num_cols = ["price", "beds", "bath", "propertysqft"]
for col in num_cols:
    if col in df_house.columns:
        df_house[col] = pd.to_numeric(df_house[col], errors="coerce")

df_house[num_cols].describe()


,price,beds,bath,propertysqft
count,4.801000e+03,4801.000000,4801.000000,4801.000000
mean,2.356940e+06,3.356801,2.373861,2184.207862
std,3.135525e+07,2.602315,1.946962,2377.140894
min,2.494000e+03,1.000000,0.000000,230.000000
25%,4.990000e+05,2.000000,1.000000,1200.000000
50%,8.250000e+05,3.000000,2.000000,2184.207862
75%,1.495000e+06,4.000000,3.000000,2184.207862
max,2.147484e+09,50.000000,50.000000,65535.000000


In [10]:
# Extract 5-digit ZIP from postcode if available
if "postcode" in df_house.columns:
    df_house["postcode"] = (
        df_house["postcode"]
        .astype(str)
        .str.extract(r"(\d{5})")[0]
    )
else:
    df_house["postcode"] = np.nan  # fallback if not present


In [11]:
# Create a clean 'borough' column
if "administrative_area_level_2" in df_house.columns:
    df_house["borough"] = (
        df_house["administrative_area_level_2"]
        .astype(str)
        .str.strip()
        .str.upper()
    )
else:
    # If you already have a borough-like column, adjust here
    df_house["borough"] = np.nan

df_house["borough"].value_counts(dropna=False).head()


borough
UNITED STATES      2484
NEW YORK           2241
NEW YORK COUNTY      21
11214                 8
QUEENS COUNTY         6
Name: count, dtype: int64

In [36]:
df_house["borough"] = df_house["borough"].astype(str).str.upper().str.strip()


In [12]:
borough_map = {
    "NEW YORK COUNTY": "MANHATTAN",
    "KINGS COUNTY": "BROOKLYN",
    "BRONX COUNTY": "BRONX",
    "RICHMOND COUNTY": "STATEN ISLAND",
    "QUEENS COUNTY": "QUEENS",
}

df_house["borough"] = df_house["borough"].replace(borough_map)


In [13]:
df_house["borough"] = df_house["borough"].replace({
    "UNITED STATES": np.nan,
    "NEW YORK": np.nan
})


In [14]:
zip_to_borough = {
    "100": "MANHATTAN", "101": "MANHATTAN", "102": "MANHATTAN",
    "103": "STATEN ISLAND",
    "104": "BRONX",
    "112": "BROOKLYN",
    "111": "QUEENS", "113": "QUEENS", "114": "QUEENS", "116": "QUEENS",
}

def assign_borough_from_zip(zipcode):
    if pd.isna(zipcode):
        return np.nan
    prefix = str(zipcode)[:3]
    return zip_to_borough.get(prefix, np.nan)

df_house["borough_fixed"] = df_house.apply(
    lambda row: assign_borough_from_zip(row["postcode"]) 
                if pd.isna(row["borough"]) or row["borough"] in ["UNITED STATES", "NEW YORK"]
                else row["borough"],
    axis=1
)


In [15]:
df_house["borough_fixed"].value_counts(dropna=False)


borough_fixed
NaN          4725
MANHATTAN      21
11214           8
BROOKLYN        7
QUEENS          6
BRONX           5
11229           3
10471           2
10002           2
10310           2
10003           2
10017           2
10306           2
11237           1
10466           1
10465           1
11412           1
10312           1
10301           1
10309           1
10463           1
10304           1
11417           1
11414           1
11234           1
10303           1
11218           1
Name: count, dtype: int64

In [16]:
import numpy as np

# ZIP → Borough Mapping
zip_to_borough = {
    "100": "MANHATTAN", "101": "MANHATTAN", "102": "MANHATTAN",
    "103": "STATEN ISLAND",
    "104": "BRONX",
    "110": "QUEENS", "111": "QUEENS", "112": "BROOKLYN",
    "113": "QUEENS", "114": "QUEENS", "116": "QUEENS"
}

def fix_borough(row):
    b = str(row["borough"]).upper().strip() if pd.notna(row["borough"]) else ""
    z = str(row["postcode"])[:3] if pd.notna(row["postcode"]) else ""

    # If borough is clearly valid, keep it
    if b in ["MANHATTAN", "BROOKLYN", "QUEENS", "BRONX", "STATEN ISLAND"]:
        return b
    
    # If borough looks like a ZIP code (e.g., "11214"), convert using ZIP prefix
    if b[:3].isdigit():
        prefix = b[:3]
        if prefix in zip_to_borough:
            return zip_to_borough[prefix]
    
    # If borough is missing or garbage, use postcode → borough mapping
    if z.isdigit() and z in zip_to_borough:
        return zip_to_borough[z]

    return np.nan

df_house["borough_clean"] = df_house.apply(fix_borough, axis=1)


In [42]:
df_house["borough_clean"].value_counts(dropna=False)


borough_clean
NaN              4725
MANHATTAN          27
BROOKLYN           21
BRONX              10
STATEN ISLAND       9
QUEENS              9
Name: count, dtype: int64

In [17]:
df_house.columns


Index(['brokertitle', 'type', 'price', 'beds', 'bath', 'propertysqft',
       'address', 'state', 'main_address', 'administrative_area_level_2',
       'locality', 'sublocality', 'street_name', 'long_name',
       'formatted_address', 'latitude', 'longitude', 'postcode', 'borough',
       'borough_fixed', 'borough_clean'],
      dtype='object')

In [18]:
df_house["zip"] = df_house["formatted_address"].astype(str).str.extract(r"(\d{5})")

In [19]:
df_house["zip"].head(10)


0    10022
1    10019
2    10312
3    10022
4    10065
5    11238
6    10027
7    10304
8    10473
9    11230
Name: zip, dtype: object

In [20]:
zip_to_borough = {
    "100": "MANHATTAN",
    "101": "MANHATTAN",
    "102": "MANHATTAN",
    "103": "STATEN ISLAND",
    "104": "BRONX",
    "110": "QUEENS",
    "111": "QUEENS",
    "112": "BROOKLYN",
    "113": "QUEENS",
    "114": "QUEENS",
    "116": "QUEENS",
}

df_house["borough_final"] = df_house["zip"].str[:3].map(zip_to_borough)


In [21]:
df_house["borough_final"].value_counts(dropna=False)


borough_final
MANHATTAN        1295
BROOKLYN         1210
QUEENS           1167
STATEN ISLAND     549
BRONX             503
NaN                77
Name: count, dtype: int64

In [22]:
##Clean & prepare the MTA subway stations dataset
# Standardize columns
df_mta.columns = df_mta.columns.str.lower().str.strip().str.replace(" ", "_")

df_mta.head()
df_mta.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 496 entries, 0 to 495
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   gtfs_stop_id           496 non-null    object 
 1   station_id             496 non-null    int64  
 2   complex_id             496 non-null    int64  
 3   division               496 non-null    object 
 4   line                   496 non-null    object 
 5   stop_name              496 non-null    object 
 6   borough                496 non-null    object 
 7   cbd                    496 non-null    bool   
 8   daytime_routes         496 non-null    object 
 9   structure              496 non-null    object 
 10  gtfs_latitude          496 non-null    float64
 11  gtfs_longitude         496 non-null    float64
 12  north_direction_label  496 non-null    object 
 13  south_direction_label  496 non-null    object 
 14  ada                    496 non-null    int64  
 15  ada_no

In [51]:
# Keep only rows with valid coordinates
coord_cols = ["gtfs_latitude", "gtfs_longitude"]
df_mta = df_mta.dropna(subset=[c for c in coord_cols if c in df_mta.columns])

df_mta["gtfs_latitude"] = pd.to_numeric(df_mta["gtfs_latitude"], errors="coerce")
df_mta["gtfs_longitude"] = pd.to_numeric(df_mta["gtfs_longitude"], errors="coerce")
df_mta = df_mta.dropna(subset=["gtfs_latitude", "gtfs_longitude"])

df_mta.shape


(496, 19)

In [52]:
##Clean & prepare the MTA subway stations dataset
# Standardize columns
df_crime.columns = df_crime.columns.str.lower().str.strip().str.replace(" ", "_")

df_crime.head()
df_crime.columns


Index(['cmplnt_num', 'cmplnt_fr_dt', 'cmplnt_fr_tm', 'cmplnt_to_dt',
       'cmplnt_to_tm', 'addr_pct_cd', 'rpt_dt', 'ky_cd', 'ofns_desc', 'pd_cd',
       'pd_desc', 'crm_atpt_cptd_cd', 'law_cat_cd', 'boro_nm',
       'loc_of_occur_desc', 'prem_typ_desc', 'juris_desc', 'jurisdiction_code',
       'parks_nm', 'hadevelopt', 'housing_psa', 'x_coord_cd', 'y_coord_cd',
       'susp_age_group', 'susp_race', 'susp_sex', 'latitude', 'longitude',
       'lat_lon', 'patrol_boro', 'station_name', 'vic_age_group', 'vic_race',
       'vic_sex', ':@computed_region_efsh_h5xi', ':@computed_region_f5dn_yrer',
       ':@computed_region_yeji_bk3q', ':@computed_region_92fq_4b7q',
       ':@computed_region_sbqj_enih', 'transit_district'],
      dtype='object')

In [23]:
cols_we_want = ["boro_nm", "cmplnt_num", "law_cat_cd"]
cols_we_have = [c for c in cols_we_want if c in df_crime.columns]

df_crime_small = df_crime[cols_we_have].copy()
df_crime_small = df_crime_small.dropna(subset=["boro_nm"])

df_crime_small["boro_nm"].value_counts()


boro_nm
BROOKLYN         626
MANHATTAN        545
BRONX            372
QUEENS           340
STATEN ISLAND    109
(null)             8
Name: count, dtype: int64

In [24]:
# Aggregate crime stats by borough name
df_crime_boro = df_crime_small.groupby("boro_nm").agg(
    crime_count=("cmplnt_num", "count"),
    felony_share=("law_cat_cd", lambda x: (x == "FELONY").mean() if "FELONY" in x.values else 0.0)
).reset_index()

# Create a 'borough' column matching housing format (UPPERCASE)
df_crime_boro["borough"] = df_crime_boro["boro_nm"].str.upper()

df_crime_boro


,boro_nm,crime_count,felony_share,borough
0,(null),8,0.000000,(NULL)
1,BRONX,372,0.384409,BRONX
2,BROOKLYN,626,0.335463,BROOKLYN
3,MANHATTAN,545,0.326606,MANHATTAN
4,QUEENS,340,0.326471,QUEENS
5,STATEN ISLAND,109,0.247706,STATEN ISLAND


In [25]:
# Make sure housing borough is uppercase to match
df_house["borough"] = df_house["borough"].astype(str).str.upper()

df_house_crime = df_house.merge(
    df_crime_boro[["borough", "crime_count", "felony_share"]],
    on="borough",
    how="left"
)

df_house_crime[["borough", "crime_count", "felony_share"]].head()


,borough,crime_count,felony_share
0,MANHATTAN,545.0,0.326606
1,NAN,NaN,NaN
2,NAN,NaN,NaN
3,NAN,NaN,NaN
4,NAN,NaN,NaN


In [26]:
df_house["borough"].dropna().unique()


array(['MANHATTAN', 'NAN', 'BRONX', '11214', '10301', '10309', '10303',
       '11234', '11414', '10310', '10003', '11417', '10304', 'BROOKLYN',
       '10463', 'QUEENS', '10017', '10306', '10471', '11229', '10312',
       '11412', '10465', '10002', '10466', '11237', '11218'], dtype=object)

In [27]:


# Make sure these exist as strings
df_house["formatted_address"] = df_house["formatted_address"].astype(str)
df_house["postcode"] = df_house["postcode"].astype(str)

# Prefer ZIP from formatted_address; fall back to postcode
df_house["zip"] = (
    df_house["formatted_address"]
      .str.extract(r"(\d{5})")[0]
      .fillna(df_house["postcode"].str.extract(r"(\d{5})")[0])
)

df_house["zip"].head(10)


0    10022
1    10019
2    10312
3    10022
4    10065
5    11238
6    10027
7    10304
8    10473
9    11230
Name: zip, dtype: object

In [28]:
zip_to_borough = {
    "100": "MANHATTAN",
    "101": "MANHATTAN",
    "102": "MANHATTAN",
    "103": "STATEN ISLAND",
    "104": "BRONX",
    "110": "QUEENS",
    "111": "QUEENS",
    "112": "BROOKLYN",
    "113": "QUEENS",
    "114": "QUEENS",
    "116": "QUEENS",
}


In [29]:
valid_boros = {"MANHATTAN", "BROOKLYN", "QUEENS", "BRONX", "STATEN ISLAND"}

# Clean up existing borough column (fix 'NAN' string)
df_house["borough"] = df_house["borough"].astype(str).str.strip().str.upper()
df_house["borough"] = df_house["borough"].replace("NAN", np.nan)

def derive_borough(row):
    existing = row["borough"]
    # if already a valid borough, keep it
    if existing in valid_boros:
        return existing
    
    # otherwise try to infer from ZIP prefix
    z = row["zip"]
    if pd.notna(z):
        prefix = str(z)[:3]
        if prefix in zip_to_borough:
            return zip_to_borough[prefix]
    
    return np.nan

df_house["borough_clean"] = df_house.apply(derive_borough, axis=1)


In [30]:
df_house["borough_clean"].value_counts(dropna=False)


borough_clean
MANHATTAN        1295
BROOKLYN         1210
QUEENS           1167
STATEN ISLAND     549
BRONX             503
NaN                77
Name: count, dtype: int64

In [31]:
df_house = df_house.drop(columns=["borough"], errors="ignore")
df_house = df_house.rename(columns={"borough_clean": "borough"})


In [32]:
df_crime_boro["borough"] = df_crime_boro["borough"].astype(str).str.upper().str.strip()

df_house_crime = df_house.merge(
    df_crime_boro[["borough", "crime_count", "felony_share"]],
    on="borough",
    how="left"
)

df_house_crime[["borough", "crime_count", "felony_share"]].head(20)


,borough,crime_count,felony_share
0,MANHATTAN,545.0,0.326606
1,MANHATTAN,545.0,0.326606
2,STATEN ISLAND,109.0,0.247706
3,MANHATTAN,545.0,0.326606
4,MANHATTAN,545.0,0.326606
5,BROOKLYN,626.0,0.335463
6,MANHATTAN,545.0,0.326606
7,STATEN ISLAND,109.0,0.247706
8,BRONX,372.0,0.384409
9,BROOKLYN,626.0,0.335463


In [63]:
# Make sure housing borough is uppercase to match
df_house["borough"] = df_house["borough"].astype(str).str.upper()

df_house_crime = df_house.merge(
    df_crime_boro[["borough", "crime_count", "felony_share"]],
    on="borough",
    how="left"
)

df_house_crime[["borough", "crime_count", "felony_share"]].head()


,borough,crime_count,felony_share
0,MANHATTAN,545.0,0.326606
1,MANHATTAN,545.0,0.326606
2,STATEN ISLAND,109.0,0.247706
3,MANHATTAN,545.0,0.326606
4,MANHATTAN,545.0,0.326606


In [46]:
df_house_crime[["price","crime_count"]].corr()


,price,crime_count
price,1.00000,-0.00667
crime_count,-0.00667,1.00000


In [47]:
df_house_crime[["price","felony_share"]].corr()


,price,felony_share
price,1.000000,-0.032932
felony_share,-0.032932,1.000000


In [50]:
df_house_crime["crime_level"] = pd.qcut(
    df_house_crime["crime_count"],
    q=3,
    labels=["Low Crime","Medium Crime","High Crime"]
)



In [51]:
import scipy.stats as stats

groups = [
    df_house_crime.loc[df_house_crime["crime_level"]=="Low Crime","price"],
    df_house_crime.loc[df_house_crime["crime_level"]=="Medium Crime","price"],
    df_house_crime.loc[df_house_crime["crime_level"]=="High Crime","price"]
]

stats.f_oneway(*groups)



F_onewayResult(statistic=1.3423884660286254, pvalue=0.26132069473634845)

In [34]:
df_mta = pd.read_csv("MTA_Subway_Stations_20251120.csv")
df_mta.columns



Index(['GTFS Stop ID', 'Station ID', 'Complex ID', 'Division', 'Line',
       'Stop Name', 'Borough', 'CBD', 'Daytime Routes', 'Structure',
       'GTFS Latitude', 'GTFS Longitude', 'North Direction Label',
       'South Direction Label', 'ADA', 'ADA Northbound', 'ADA Southbound',
       'ADA Notes', 'Georeference'],
      dtype='object')

In [35]:
# Standardize MTA columns -> snake_case
df_mta.columns = (df_mta.columns
                  .str.strip()
                  .str.lower()
                  .str.replace(" ", "_"))

# You should now have 'gtfs_latitude' and 'gtfs_longitude'
assert "gtfs_latitude" in df_mta.columns and "gtfs_longitude" in df_mta.columns, df_mta.columns

# Keep valid station coordinates
df_mta["gtfs_latitude"]  = pd.to_numeric(df_mta["gtfs_latitude"], errors="coerce")
df_mta["gtfs_longitude"] = pd.to_numeric(df_mta["gtfs_longitude"], errors="coerce")
df_mta = df_mta.dropna(subset=["gtfs_latitude","gtfs_longitude"]).reset_index(drop=True)

# Ensure housing coords are numeric & not null
for c in ["latitude","longitude"]:
    df_house[c] = pd.to_numeric(df_house[c], errors="coerce")
df_house = df_house.dropna(subset=["latitude","longitude"]).reset_index(drop=True)

# Build station arrays once
stations_lat = df_mta["gtfs_latitude"].to_numpy()
stations_lon = df_mta["gtfs_longitude"].to_numpy()

In [36]:
# Haversine distance counter (meters)
def count_stations_within(lat, lon, thresh_m=800):
    lat1 = np.radians(lat)
    lon1 = np.radians(lon)
    lat2 = np.radians(stations_lat)
    lon2 = np.radians(stations_lon)
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2.0)**2 + np.cos(lat1)*np.cos(lat2)*np.sin(dlon/2.0)**2
    c = 2*np.arctan2(np.sqrt(a), np.sqrt(1-a))
    dist_m = 6371000.0 * c
    return np.count_nonzero(dist_m <= thresh_m)

In [37]:
sample = df_house.sample(min(50, len(df_house)), random_state=42).copy()
sample["stations_within_800m"] = sample.apply(
    lambda r: count_stations_within(r["latitude"], r["longitude"], 800), axis=1
)
print(sample[["borough","price","stations_within_800m"]].head(10))

            borough    price  stations_within_800m
596   STATEN ISLAND   750000                     1
4507         CORONA   899000                     0
3049      MANHATTAN  5900000                     5
2957       BROOKLYN  1695000                     3
8             BRONX   265000                     0
577        BROOKLYN  3795000                     3
3564      MANHATTAN  2400000                     3
811   STATEN ISLAND   778000                     0
4638          BRONX   999000                     1
1538      MANHATTAN   789000                     2


In [38]:
# Now let's compute for all rows
df_house["stations_within_800m"] = df_house.apply(
    lambda r: count_stations_within(r["latitude"], r["longitude"], 800), axis=1
)

# Quick sanity checks
print("\nMedian stations within 800m by borough:")
print(df_house.groupby("borough")["stations_within_800m"].median().sort_values(ascending=False))

print("\nCorrelation (price vs stations_within_800m):")
print(df_house[["price","stations_within_800m"]].corr().iloc[0,1])


Median stations within 800m by borough:
borough
BROOKLYN HEIGHTS    7.0
BROWNVILLE          5.0
BRONX NY            4.0
MANHATTAN           4.0
LONG ISLAND CITY    3.0
                   ... 
MIDDLE VILLAGE      0.0
MASPETH             0.0
DOUGLASTON          0.0
MALBA               0.0
FLORAL PARK         0.0
Name: stations_within_800m, Length: 70, dtype: float64

Correlation (price vs stations_within_800m):
0.03743111392695638


In [39]:
# Extract ZIP from formatted_address
df_house["zip"] = df_house["formatted_address"].str.extract(r"(\d{5})")

# NYC ZIP → borough mapping
zip_to_boro = {
    "100": "MANHATTAN", "101": "MANHATTAN", "102": "MANHATTAN",
    "103": "STATEN ISLAND",
    "104": "BRONX",
    "110": "QUEENS",
    "111": "QUEENS",
    "112": "BROOKLYN",
    "113": "QUEENS",
    "114": "QUEENS",
    "116": "QUEENS",
}

df_house["borough"] = df_house["zip"].str[:3].map(zip_to_boro)


In [40]:
df_house["borough"].value_counts(dropna=False)


borough
MANHATTAN        1295
BROOKLYN         1210
QUEENS           1167
STATEN ISLAND     549
BRONX             503
NaN                77
Name: count, dtype: int64

In [41]:
df_house.groupby("borough")["stations_within_800m"].median()


borough
BRONX            1.0
BROOKLYN         2.0
MANHATTAN        4.0
QUEENS           0.0
STATEN ISLAND    0.0
Name: stations_within_800m, dtype: float64

In [42]:
df_house[["price","stations_within_800m"]].corr()


,price,stations_within_800m
price,1.000000,0.037431
stations_within_800m,0.037431,1.000000


In [43]:
df_house.groupby("borough")[["price","stations_within_800m"]].corr().iloc[0::2,-1]


borough             
BRONX          price   -0.045522
BROOKLYN       price    0.182235
MANHATTAN      price    0.090699
QUEENS         price   -0.058485
STATEN ISLAND  price    0.095339
Name: stations_within_800m, dtype: float64

In [44]:
df_house["transit_level"] = pd.cut(
    df_house["stations_within_800m"],
    bins=[-1,0,3,7,20],
    labels=["No Transit","Low Transit","Medium Transit","High Transit"]
)


In [45]:
import scipy.stats as stats

groups = [
    df_house.loc[df_house["transit_level"]=="No Transit","price"],
    df_house.loc[df_house["transit_level"]=="Low Transit","price"],
    df_house.loc[df_house["transit_level"]=="Medium Transit","price"],
    df_house.loc[df_house["transit_level"]=="High Transit","price"]
]

stats.f_oneway(*groups)


F_onewayResult(statistic=2.4392877738405425, pvalue=0.06256168350448853)

In [52]:
import numpy as np
import pandas as pd

import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.stats.diagnostic import het_breuschpagan
from patsy import dmatrices

import matplotlib.pyplot as plt


In [54]:
# Ensure transit column exists
if "stations_within_800m" not in df_house_crime.columns:
    print("Adding missing 'stations_within_800m' from df_house...")
    
    # Find a safe join key
    if "formatted_address" in df_house.columns:
        key = "formatted_address"
    elif "address" in df_house.columns:
        key = "address"
    else:
        raise KeyError("No shared key found to merge stations_within_800m")

    df_house_crime = df_house_crime.merge(
        df_house[[key, "stations_within_800m"]],
        on=key,
        how="left"
    )

print("'stations_within_800m' now present:", "stations_within_800m" in df_house_crime.columns)


Adding missing 'stations_within_800m' from df_house...
'stations_within_800m' now present: True


In [55]:
# If you already have df_house_crime from your merge, keep this True.
HAVE_MERGED = 'df_house_crime' in globals()

if not HAVE_MERGED:
    # Minimal rebuild if needed (expects df_house, df_crime_boro, and stations_within_800m on df_house)
    df_crime_boro["borough"] = df_crime_boro["borough"].str.upper().str.strip()
    df_house["borough"] = df_house["borough"].str.upper().str.strip()
    df_house_crime = df_house.merge(
        df_crime_boro[["borough", "crime_count", "felony_share"]],
        on="borough",
        how="left"
    )

# Keep only columns we need; coerce numerics and drop obvious outliers
use_cols = [
    "price", "propertysqft", "beds", "bath",
    "stations_within_800m", "crime_count", "felony_share", "borough"
]
dfm = df_house_crime[use_cols].copy()

# Make numeric
num_cols = ["price","propertysqft","beds","bath","stations_within_800m","crime_count","felony_share"]
for c in num_cols:
    dfm[c] = pd.to_numeric(dfm[c], errors="coerce")

# Basic sanity filters (adjust if needed)
dfm = dfm[
    (dfm["price"].between(25_000, 60_000_000)) &
    (dfm["propertysqft"].between(100, 20_000)) &
    (dfm["beds"].between(0, 12)) &
    (dfm["bath"].between(0, 12))
]

# Drop rows missing key fields
dfm = dfm.dropna(subset=["price","propertysqft","beds","bath","stations_within_800m","crime_count","felony_share","borough"]).copy()

# Ensure borough is one of the 5
valid_boros = {"MANHATTAN","BROOKLYN","QUEENS","BRONX","STATEN ISLAND"}
dfm = dfm[dfm["borough"].isin(valid_boros)].copy()

print(dfm.shape)
dfm.head()


(5182, 8)


,price,propertysqft,beds,bath,stations_within_800m,crime_count,felony_share,borough
0,315000,1400.0,2,2.000000,12,545.0,0.326606,MANHATTAN
2,260000,2015.0,4,2.000000,0,109.0,0.247706,STATEN ISLAND
3,69000,445.0,3,1.000000,11,545.0,0.326606,MANHATTAN
4,69000,445.0,3,1.000000,11,545.0,0.326606,MANHATTAN
5,55000000,14175.0,7,2.373861,6,545.0,0.326606,MANHATTAN


In [56]:
# Log-transform price to stabilize skew and interpret % changes
dfm["log_price"] = np.log(dfm["price"])

# Optionally scale felony_share to percentage for readability in interpretation
dfm["felony_pct"] = dfm["felony_share"] * 100

# Patsy formula with borough fixed effects (dummy variables)
# Model will estimate effects relative to the first category (alphabetical unless re-ordered)
formula_lin  = "price ~ propertysqft + beds + bath + stations_within_800m + crime_count + felony_pct + C(borough)"
formula_log  = "log_price ~ propertysqft + beds + bath + stations_within_800m + crime_count + felony_pct + C(borough)"

y_lin,  X_lin  = dmatrices(formula_lin, data=dfm, return_type="dataframe")
y_log,  X_log  = dmatrices(formula_log, data=dfm, return_type="dataframe")

# Peek columns
X_log.columns


Index(['Intercept', 'C(borough)[T.BROOKLYN]', 'C(borough)[T.MANHATTAN]',
       'C(borough)[T.QUEENS]', 'C(borough)[T.STATEN ISLAND]', 'propertysqft',
       'beds', 'bath', 'stations_within_800m', 'crime_count', 'felony_pct'],
      dtype='object')

In [57]:
# Plain OLS
m_lin = sm.OLS(y_lin, X_lin).fit()
m_log = sm.OLS(y_log, X_log).fit()

print("=== Linear price model ===")
print(m_lin.summary())

print("\n\n=== Log(price) model (recommended) ===")
print(m_log.summary())


=== Linear price model ===
                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.430
Model:                            OLS   Adj. R-squared:                  0.430
Method:                 Least Squares   F-statistic:                     488.6
Date:                Sat, 22 Nov 2025   Prob (F-statistic):               0.00
Time:                        18:29:24   Log-Likelihood:                -84520.
No. Observations:                5182   AIC:                         1.691e+05
Df Residuals:                    5173   BIC:                         1.691e+05
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                                  coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------

In [11]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(10,6))
sns.heatmap(
    df_house_crime[["price","propertysqft","beds","bath",
                    "stations_within_800m","crime_count","felony_share"]]
    .corr(),
    annot=True,
    cmap="coolwarm"
)
plt.title("Correlation Heatmap of Housing Features")
plt.show()


NameError: name 'df_house_crime' is not defined

<Figure size 1000x600 with 0 Axes>